## Routewise Average Delay Performance Statistics
What is the average delay, number of cancellations, number of diverted flights for a given route for all airlines for a particular month?

1. Please enter Origin, Destination and Month Details on Top

In [3]:
#Enter Details on top of the page 

dbutils.widgets.text("Month", "Enter the month")
dbutils.widgets.text("Origin", "Enter the Origin")
dbutils.widgets.text("Destination", "Enter the Destination")

In [4]:
import pandas as pd

#Function to find average delay information per route for every airline
def find_route_delay_info(Origin,Dest,Airline):
  
  print(' The route is Origin :', Origin, ' and the Destination is :', Dest)

  #Load table into spark dataframe
  airline = spark.table("airline_2018_19_final")
  
  #Obtain carrier name information
  airport_code = spark.table("CARRIERS").toPandas()

  #Filter Airline df on Origin and Destination
  airline_delays_df=airline.filter((airline.ORIGIN==Origin) & (airline.DEST==Dest)).toPandas()
  
  #Obtain Airline Carrier name from Carriers table and replace Carrier ID in Airline Delay df
  airline_delays_df['OP_UNIQUE_CARRIER'] = airline_delays_df['OP_UNIQUE_CARRIER'].map(airport_code.set_index('Code')['Description'])
 
  #Find total number of flights in that route along with total cancellations and diverted flights
  g=airline_delays_df.groupby('OP_UNIQUE_CARRIER')['FLIGHTS','CANCELLED','DIVERTED'].sum()
  
  #Calculate Total Delay with the below formaula. We are not taking an average since Arrival delay has more importance than departure delay
  airline_delays_df['Total_Delay']=airline_delays_df['ARR_DELAY']*0.6 + airline_delays_df['DEP_DELAY']*0.4
 
  #Find Average Delays for each airline
  j = airline_delays_df.groupby('OP_UNIQUE_CARRIER')['Total_Delay','DEP_DELAY','ARR_DELAY','WEATHER_DELAY','CARRIER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY'].mean()
  
  #Merge the two dfs are return 
  return pd.merge(g, j, left_on='OP_UNIQUE_CARRIER', right_on='OP_UNIQUE_CARRIER')

#Function to obtain the price information for a given route and airline 
def find_price_route_airline(Date,Origin,Dest,Airline):
  
  #Find which quarter the month belongs to
  if Date<4:
    quarter=1
  elif Date<7:
    quarter=2
  elif Date<10:
    quarter=3
  else:
    quarter=4
  
  print(' The route is Origin :', Origin, ' and the Destination is :', Dest, ' for the Quarter :',quarter)
  
  airline_price = spark.table("airlines_price")
  
  #Filter based on route
  airline_price_df=airline_price.filter((airline_price.ORIGIN==Origin) & (airline_price.DEST==Dest))
  
  #Select only relevant columns
  airline_price_df=airline_price_df.select('ORIGIN','DEST','QUARTER','REPORTING_CARRIER','PASSENGERS','MARKET_FARE').toPandas()
  
  #Obtain carrier name information
  airport_code = spark.table("CARRIERS").toPandas()

  #Obtain Airline Carrier name from Carriers table and replace Carrier ID in Airline Price df
  airline_price_df['REPORTING_CARRIER'] = airline_price_df['REPORTING_CARRIER'].map(airport_code.set_index('Code')['Description'])

  airline_price_df['MARKET_FARE']=airline_price_df['MARKET_FARE'].astype('float64')
  
  #Calculate average price per quarter for each airline for given route
  return airline_price_df.groupby(['QUARTER','REPORTING_CARRIER'])['MARKET_FARE'].mean().to_frame()


In [5]:
date = int(dbutils.widgets.get("Month"))
origin=dbutils.widgets.get("Origin")
dest=dbutils.widgets.get("Destination")

#Find Average delay information for given route
find_route_delay_info(Origin=origin,Dest=dest,Airline=None)

,FLIGHTS,CANCELLED,DIVERTED,Total_Delay,DEP_DELAY,ARR_DELAY,WEATHER_DELAY,CARRIER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
OP_UNIQUE_CARRIER,,,,,,,,,,,
American Airlines Inc.,2185.0,19.0,7.0,5.417416,8.597600,3.335340,0.053412,27.694363,5.792285,0.160237,33.160236
Delta Air Lines Inc.,1613.0,2.0,4.0,1.198506,6.825574,-2.497822,0.251232,22.842365,7.211823,0.000000,23.886700
SkyWest Airlines Inc.,1110.0,3.0,5.0,-2.416334,3.059621,-6.058076,0.442308,31.423077,6.259615,0.096154,25.711538
Southwest Airlines Co.,1531.0,8.0,5.0,5.477470,11.381484,1.604743,0.000000,18.378378,3.945946,0.000000,28.208494
Spirit Air Lines,122.0,1.0,0.0,-5.262810,-1.231405,-7.950413,0.000000,36.666668,32.555557,0.000000,0.000000
United Air Lines Inc.,20.0,0.0,0.0,-19.260000,-4.950000,-28.799999,NaN,NaN,NaN,NaN,NaN


## Routewise Average Market Price for each Airline
What is the average market fare for a given route for all airlines for a particular month?

In [7]:
date = int(dbutils.widgets.get("Month"))
origin=dbutils.widgets.get("Origin")
dest=dbutils.widgets.get("Destination")

df= pd.DataFrame(find_price_route_airline(Date=date,Origin=origin,Dest=dest,Airline=None).reset_index())

df

,QUARTER,REPORTING_CARRIER,MARKET_FARE
0,1,Alaska Airlines Inc.,359.800000
1,1,American Airlines Inc.,257.558899
2,1,Compass Airlines,298.780588
3,1,Delta Air Lines Inc.,271.826603
4,1,Envoy Air,336.040000
5,1,ExpressJet Airlines LLC,254.140000
6,1,Frontier Airlines Inc.,96.300000
7,1,GoJet Airlines LLC d/b/a United Express,187.800000
8,1,Mesa Airlines Inc.,5.020000
9,1,SkyWest Airlines Inc.,239.707921


#### Obtain Barplot of Price Trend for each Quarter for each Airline flying that route

In [9]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

a4_dims = (16, 10)
fig, ax = plt.subplots(figsize=a4_dims)
ax = sns.barplot(x='QUARTER', y="MARKET_FARE", hue='REPORTING_CARRIER',data=df)
display(ax)

#### Obtain Price Trend for each Quarter for each Airline flying that route

In [11]:
a4_dims = (8, 5)
fig, ax = plt.subplots(figsize=a4_dims)
ax = sns.lineplot(x='QUARTER', y="MARKET_FARE",markers=True, dashes=True, data=df)
display(ax)